In [36]:
#parse cma data
config_file = "/home/bas/data/dt_grid_v2.csv"
performance_data = "/home/bas/data/Performance_tables_modcma" #conf_0_5D.csv

import os
import pandas as pd
import re
import numpy as np
from tqdm import tqdm

configs = pd.read_csv(config_file)

powers = np.round(np.linspace(8, -8, 81), decimals=1)
target_values = np.power([10] * 81, powers)

def calc_auc(aucs, runtime_dif, y):
    #calculate auc
    auc_changes = sum(y > target_values) / 81
    aucs -= runtime_dif*(auc_changes)
    return aucs

# iterate over files in
# that directory
features = ['elitist', 'mirrored', 'base_sampler', 'weights_option', 'local_restart', 'step_size_adaptation']
new_data = pd.DataFrame(columns = ["fid", "iid", "dim", "seed", *features , "auc"])
for filename in tqdm(os.listdir(performance_data)):
    if "attrs" not in filename:
        f = os.path.join(performance_data, filename)
        
        # checking if it is a file
        if os.path.isfile(f):
            df = pd.read_csv(f)
            z = re.match("conf_(\d+)_(\d+?)D.csv", filename)
            if z:
                #function, alg, iid, budget, auc
                conf_id = int(z.group(1))
                dim = int(z.group(2))
            aucs = [10000] * 10 #calculate for budget = 10000
            prev_runtime = 0
            first_time = True
            for row_id in range(len(df)):
                row = df.loc[row_id]
                if row["runtime"] == 500:
                    if not first_time:
                        for seed in range(10):
                            new_item =  {"fid": fid, "iid": iid, "dim": dim, "seed" : seed, "auc" : aucs[seed]}
                            for feat in features:
                                new_item[feat] = configs.loc[conf_id][feat]
                            new_data.loc[len(new_data)] = new_item
                    #new one
                    first_time = False
                    aucs = [10000] * 10 #reset auc
                    prev_runtime = 0
                    fid = row['funcId']
                    iid = row["ID"]
                    #"run.1","run.2","run.3","run.4","run.5","run.6","run.7","run.8","run.9","run.10"
                elif row["runtime"] > 10000:
                    continue
                #calc aucs
                for seed in range(10):
                    aucs[seed] = calc_auc(aucs[seed], row["runtime"] - prev_runtime, row[f"run.{(seed+1)}"])
                    
                prev_runtime = row["runtime"]

new_data.to_pickle("cma_es.pkl")
new_data


100%|██████████| 1296/1296 [3:36:35<00:00, 10.03s/it]  


,fid,iid,dim,seed,elitist,mirrored,base_sampler,weights_option,local_restart,step_size_adaptation,auc
0,1,1,30,0,True,mirrored pairwise,sobol,default,NaN,csa,9302.469136
1,1,1,30,1,True,mirrored pairwise,sobol,default,NaN,csa,9296.296296
2,1,1,30,2,True,mirrored pairwise,sobol,default,NaN,csa,9320.987654
3,1,1,30,3,True,mirrored pairwise,sobol,default,NaN,csa,9296.296296
4,1,1,30,4,True,mirrored pairwise,sobol,default,NaN,csa,9320.987654
...,...,...,...,...,...,...,...,...,...,...,...
771115,8,2,30,5,False,NaN,gaussian,default,NaN,csa,4018.518519
771116,8,2,30,6,False,NaN,gaussian,default,NaN,csa,4055.555556
771117,8,2,30,7,False,NaN,gaussian,default,NaN,csa,4129.629630
771118,8,2,30,8,False,NaN,gaussian,default,NaN,csa,4104.938272


In [1]:
#parse cma data
config_file = "/home/bas/data/MOD_DE/dt_grid_de.csv"
performance_data = "/home/bas/data/MOD_DE/data/" #conf_0_5D.csv
performance_data_i = "/home/bas/data/MOD_DE/data/Datatables_DE_I5/"

import os
import pandas as pd
import re
import numpy as np
from tqdm import tqdm

configs = pd.read_csv(config_file)
use_seeds = 10
powers = np.round(np.linspace(8, -8, 81), decimals=1)
target_values = np.power([10] * 81, powers)

def calc_auc(aucs, runtime_dif, y):
    #calculate auc
    auc_changes = sum(y > target_values) / 81
    aucs -= runtime_dif*(auc_changes)
    return aucs

# iterate over files in
# that directory
features = ['mutation_base','mutation_reference','mutation_n_comps','use_archive','crossover','adaptation_method','lpsr']
new_data = pd.DataFrame(columns = ["fid", "iid", "dim", "seed", *features , "auc"])

for filename in tqdm(os.listdir(performance_data)):
    if "attrs" not in filename:
        f = os.path.join(performance_data, filename)
        
        # checking if it is a file
        if os.path.isfile(f):
            df = pd.read_csv(f)
            z = re.match("conf_(\d+)_(\d+?)D.csv", filename)
            if z:
                #function, alg, iid, budget, auc
                conf_id = int(z.group(1))
                dim = int(z.group(2))
            aucs = [10000] * use_seeds #calculate for budget = 10000
            prev_runtime = 0
            first_time = True
            for row_id in range(len(df)):
                row = df.loc[row_id]
                if row["runtime"] == 500:
                    if not first_time:
                        for seed in range(use_seeds):
                            new_item =  {"fid": fid, "iid": iid, "dim": dim, "seed" : seed, "auc" : aucs[seed]}
                            for feat in features:
                                new_item[feat] = configs.loc[conf_id][feat]
                            new_data.loc[len(new_data)] = new_item
                    #new one
                    first_time = False
                    aucs = [10000] * use_seeds #reset auc
                    prev_runtime = 0
                    fid = row['funcId']
                    iid = row["ID"]
                    #"run.1","run.2","run.3","run.4","run.5","run.6","run.7","run.8","run.9","run.10"
                elif row["runtime"] > 10000:
                    continue
                #calc aucs
                for seed in range(use_seeds):
                    try:
                        aucs[seed] = calc_auc(aucs[seed], row["runtime"] - prev_runtime, row[f"run.{(seed+1)}"])
                    except:
                        print(filename, row)

                    
                prev_runtime = row["runtime"]


for filename in tqdm(os.listdir(performance_data_i)):
    if "attrs" not in filename:
        f = os.path.join(performance_data_i, filename)
        
        # checking if it is a file
        if os.path.isfile(f):
            df = pd.read_csv(f)
            z = re.match("conf_(\d+)_(\d+?)D.csv", filename)
            if z:
                #function, alg, iid, budget, auc
                conf_id = int(z.group(1))
                dim = int(z.group(2))
            aucs = [10000] * use_seeds #calculate for budget = 10000
            prev_runtime = 0
            first_time = True
            for row_id in range(len(df)):
                row = df.loc[row_id]
                if row["runtime"] == 500:
                    if not first_time:
                        for seed in range(use_seeds):
                            new_item =  {"fid": fid, "iid": iid, "dim": dim, "seed" : seed, "auc" : aucs[seed]}
                            for feat in features:
                                new_item[feat] = configs.loc[conf_id][feat]
                            new_data.loc[len(new_data)] = new_item
                    #new one
                    first_time = False
                    aucs = [10000] * use_seeds #reset auc
                    prev_runtime = 0
                    fid = row['funcId']
                    iid = row["ID"]
                    #"run.1","run.2","run.3","run.4","run.5","run.6","run.7","run.8","run.9","run.10"
                elif row["runtime"] > 10000:
                    continue
                #calc aucs
                for seed in range(use_seeds):
                    try:
                        aucs[seed] = calc_auc(aucs[seed], row["runtime"] - prev_runtime, row[f"run.{(seed+1)}"])
                    except:
                        print(filename, row)

                    
                prev_runtime = row["runtime"]

new_data.to_pickle("mod_de_all.pkl")
new_data


100%|██████████| 1152/1152 [1:54:20<00:00,  5.96s/it]


,fid,iid,dim,seed,mutation_base,mutation_reference,mutation_n_comps,use_archive,crossover,adaptation_method,lpsr,auc
0,1,1,30,0,target,pbest,1,True,exp,NaN,True,4265.432099
1,1,1,30,1,target,pbest,1,True,exp,NaN,True,4240.740741
2,1,1,30,2,target,pbest,1,True,exp,NaN,True,4222.222222
3,1,1,30,3,target,pbest,1,True,exp,NaN,True,4179.012346
4,1,1,30,4,target,pbest,1,True,exp,NaN,True,4197.530864
...,...,...,...,...,...,...,...,...,...,...,...,...
1359355,8,5,5,5,target,pbest,2,False,exp,shade,False,8796.296296
1359356,8,5,5,6,target,pbest,2,False,exp,shade,False,9129.629630
1359357,8,5,5,7,target,pbest,2,False,exp,shade,False,8685.185185
1359358,8,5,5,8,target,pbest,2,False,exp,shade,False,8006.172840


In [1]:
from ioh_xplainer import explainer
import pandas as pd

from ConfigSpace import ConfigurationSpace
from ConfigSpace.util import generate_grid

cs = ConfigurationSpace({
    "F": (0.05, 2.0),              # Uniform float
    "CR" : (0.05, 2.0),                    # Uniform float
})


de_explainer = explainer(None, 
                 cs , 
                 dims = [5,30],#, 10, 20, 40 
                 fids = np.arange(1,25), #,5
                 iids = 2, #20 
                 reps = 50, 
                 sampling_method = "grid",  #or random
                 grid_steps_dict = {},
                 sample_size = None,  #only used with random method
                 budget = 10000, #10000
                 seed = 1,
                 verbose = True)


de_explainer.load_results("mod_de.pkl")
df = de_explainer.performance_stats()
print(df)
#print(de_explainer.stats[5])
#df = de_explainer.stats[5]
print(df.to_latex(index=False,
                  float_format="{:.2f}".format,
                  multicolumn_format = "c"
))  

/home/neocortex/repos/ioh-explainer/env/lib/python3.8/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/neocortex/repos/ioh-explainer/env/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/neocortex/repos/ioh-explainer/env/lib/python3.8/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/neocortex/repos/ioh-explainer/env/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
Using `tqdm.autonotebook.tqdm` in note

AttributeError: 'explainer' object has no attribute 'avg_stats'